## Topics Categorization with LDA

## reference: [How we Changed Unsupervised LDA to Semi-Supervised GuidedLDA](https://www.freecodecamp.org/news/how-we-changed-unsupervised-lda-to-semi-supervised-guidedlda-e36a95f3a164/)
### code：[GuidedLDA: Guided Topic modeling with latent Dirichlet allocation](https://github.com/vi3k6i5/GuidedLDA)

In [116]:
seed_topic_list = [['configuration', 'network drive','cisco', 'vpn','firewall','active directory'],
                   ['false_positive', 'url reclassification', 'whitelist', 'white listing','block','unauthorized encryption'],
                   ['sa offline', 'corruption','behavior monitor','web console'],
                   ['compatibility','3rd party','update', 'mcafee', 'msa', 'exchange server','quickbook','office'],
                   ['performance','high_cpu', 'crash','hang','slowness','office','bsod','3rd party'],
                   ['license', 'activation','clp','expire', 'renew', 'activate','invalid','merge','account', 'password'],
                   ['document', 'kb', 'bpg', 'article','password'],
                   ['update', 'agent','hotfix','patch','migrate'],
                   ['deployment', 'removal', 'server', 'agent','install','uninstall','reinstall','cut'],
                   ['threat','virus', 'scan', 'malware', 'detect', 'infection','ransom']
                  ]

## convert seed_topic_list and calculate no. of topics

In [117]:
extra_keyword_list = []
for i in range(len(seed_topic_list)):
    for j in range(len(seed_topic_list[i])):
        seed_topic_list[i][j]=seed_topic_list[i][j].strip()
        if ' ' in seed_topic_list[i][j]:
            extra_keyword_list.append(seed_topic_list[i][j])
            seed_topic_list[i][j]=seed_topic_list[i][j].replace(' ', '_')

extra_keyword_list=list(set(extra_keyword_list))            
n_topics=len(seed_topic_list)
n_topics, len(extra_keyword_list)

(10, 10)

## Load List of Microsoft software and Ubuntu Glossaries

In [118]:
import pickle
import os
import scrape_MS_Linux_keywords

#if not os.path.exists("keyword_list.pickle"):
scrape_MS_Linux_keywords.scrap_keywords()
    
with open("keyword_list.pickle", 'rb') as f:
    keyword_list_TERM = pickle.load(f)


In [119]:
len(keyword_list_TERM.keys())

703

## Add extra keywords into keyword_list

In [120]:
# add extra keywords into keyword_list
for item in extra_keyword_list:
    if not item in keyword_list_TERM:
        keyword_list_TERM[item]={'converted_word': item.replace(' ','_'), 'count': 1}        

# save back
with open("keyword_list.pickle", 'wb') as f:
    pickle.dump(keyword_list_TERM, f)        

In [121]:
# test data
import pandas as pd
df = pd.read_csv('./WF Cases 20190517.csv')
df.head(10)

,Case Number,Subject,Description,Product Name,Product Version,Date/Time Opened,Solution Provided
0,00677507,[WFBS-SVC] Installation on Mac Is Not Full Fea...,NaN,Worry-Free Business Security Services,NaN,1/1/2018 12:18 AM,NaN
1,00677513,[WFBS - A] Smart Scan is not updating,CRC proc crash instability unable to keep syst...,Worry-Free Business Security Advanced,9.5,1/1/2018 2:30 AM,NaN
2,00677518,[WFBS - SVC] Installation issue,"Migrated a client to a new computer, the softw...",Worry-Free Business Security Services,6.2 (Beta),1/1/2018 6:41 AM,NaN
3,00677669,[MALWARE][WFBS SVC] Wallet Ransomware,NaN,Worry-Free Business Security Services,NaN,1/1/2018 11:54 PM,NaN
4,00677682,[MALWARE][WFBS S 9.5]Possible Ransomware detec...,OMAN wfbs SYED AIJAZ LOAY INTERNATIONAL LLC ai...,Worry-Free Business Security Standard,9.5,1/2/2018 12:30 AM,Next Action Plan: - check detection on the con...
5,00677799,[WFBS-S] Adding Exclusion to Behavior Monitoring,[Case Description] - client SWA financial plan...,Worry-Free Business Security Standard,NaN,1/2/2018 10:37 AM,NaN
6,00677803,[WFBS-S] Migrating from An Old Server to A New...,Issue: - customer wants to migrate WFBS to new...,Worry-Free Business Security Standard,NaN,1/2/2018 10:49 AM,NaN
7,00677816,[WFBS-SVC] Agent cloning,Can Worry Free agent with OS be image cloned/g...,Worry-Free Business Security Services,6.2,1/2/2018 12:02 PM,- manually reset the GUID info on the registry
8,00677831,[WFBS-SVC] Main Software not Functioning with ...,The latest Trend Micro update prevents one of ...,Worry-Free Business Security Services,6.2 (Beta),1/2/2018 1:13 PM,NaN
9,00677857,[WFBS-SVC] False Positive: Application Detecte...,[Issue]: False Positive: Application Detected ...,Worry-Free Business Security Standard,NaN,1/2/2018 2:38 PM,NaN


In [122]:
df[df.Subject.str.lower().isin(extra_keyword_list)]

,Case Number,Subject,Description,Product Name,Product Version,Date/Time Opened,Solution Provided
10626,00874657,URL Reclassification,NaN,Worry-Free Business Security Services,NaN,5/25/2018 12:52 AM,NaN
29475,01732305,url reclassification,NaN,Worry-Free Business Security Services,5.7,2/21/2019 1:53 AM,NaN


In [123]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
#from nltk.stem.porter import PorterStemmer
#from nltk.stem.snowball import SnowballStemmer

import krovetz
from nltk.tokenize import word_tokenize

stop_words=stopwords.words('english')
break_words=list('\n\t')
prefix_char_remove=list('.*,-')

stemmer = krovetz.PyKrovetzStemmer()
#stemmer = SnowballStemmer("english")
#stemmer.stem('working')
#lemmatizer = WordNetLemmatizer()
word_freqs = dict()


## 以 regular expression 去除 email、數字、URL、Phone No、序號

In [124]:
re_pattern_list=[]
re_pattern_list.append('[a-zA-Z0-9_]+\.(com|org|net)[a-zA-Z0-9_\.]*$') #email
re_pattern_list.append('[0-9_]+') # all digits
re_pattern_list.append("\/\/[\w.-]+(?:\.[\w\.-]+)+[\w\-\._~:/?#[\]@!\$&'\(\)\*\+,;=.]+$") # url
re_pattern_list.append('[0-9\-]+') # phone no
re_pattern_list.append('[\u0800-\u4e00]+') # 日文
re_pattern_list.append('(trend|trend\smicro)') # trend micro


## 字詞清理函數

In [125]:
# clean words
def clean_word(word_0):
    word_list=[]
    for word in word_0.split('/'):

        if len(word) <= 0 :
            continue

        # remove 網路芳鄰路徑
        remote_path=word.split('\\')
        if len(remote_path) > 0 and len(remote_path[-1])>0:
            word=remote_path[-1]
        elif len(remote_path) > 1 and len(remote_path[-2])>0:
            word=remote_path[-2]
        elif len(remote_path) > 2 and len(remote_path[-3])>0:
            word=remote_path[-3]

        if word[0] == '"' and word[-1] == '"':
            word=word[1:-1]
        if word[0] == "'" and word[-1] == "'":
            word=word[1:-1]
        if len(word) <= 0 :
            continue

        # repeat 3 times to remove continous chars
        if word[0] in prefix_char_remove:
            word=word[1:]
        if len(word) <= 0 :
            continue
        if word[0] in prefix_char_remove:
            word=word[1:]
        if len(word) <= 0 :
            continue
        if word[0] in prefix_char_remove:
            word=word[1:]

        # lemmatize
        word=word.strip().lower()
        word=stemmer.stem(word)
        if word=='':
            continue      
            
        # remove .exe and .com
        if word.endswith('.exe') or word.endswith('.com'):
            word=word[:-4]
            
        # 以 regular expression 去除 email、數字、URL、Phone No、序號
        is_re_list = False
        for pattern1 in re_pattern_list:
            result=re.findall(pattern1, word)
            if  len(result) > 0:
                is_re_list = True
                break
        if is_re_list:
            break
            
            
        # remove the words with one or two characters only
        if len(word) <= 2 :
            continue
        if not (word in stop_words):
            word_list.append(word)
               
    return word_list

## test get service_list

In [126]:
df.Subject[0]

'[WFBS-SVC] Installation on Mac Is Not Full Featured'

In [127]:
import re
pattern1 = '\[[a-zA-Z0-9_\-]+\]' # category
re.findall(pattern1, '[WFBS-SVCaaa]'+df.Subject[0])

['[WFBS-SVCaaa]', '[WFBS-SVC]']

## 分詞

In [128]:
import re
pattern1 = '\[[a-zA-Z0-9_\-\s]+\]' # service_list pattern, e.g. [WFBS-SVC]

# 清理後的 email DESCRIPTION
clean_corpus=[]
original_corpus=[]
for index, line in df.iterrows():
    clean_line=""
    line = line['Subject']
    
    # remove service_list, e.g. [WFBS-SVC]
    if type(line) == str and len(line) > 0: 
        service_list = re.findall(pattern1, line)
        for service_item in service_list:
            line = line.replace(service_item, ' ')          
                                  
    #print(line)
    for break_word in break_words:
        #print('-',len(break_word), break_word, '-')
        if not type(line) == str or len(line) <= 0: 
            break
        line = line.replace(break_word, ' ')
    if not type(line) == str or len(line) <= 0: 
        #print(type(line))
        continue
    words = word_tokenize(line) #line.lower().split(' ')
    for word_0 in words:        
        word_list = clean_word(word_0)
        for word in word_list:        
            if word in word_freqs:
                word_freqs[word] += 1
            else:
                word_freqs[word] = 1
            clean_line+=' '+word
    original_corpus.append(line)
    clean_corpus.append(clean_line.strip())
email_words=word_freqs.keys()            
print(len(email_words))

4644


In [129]:
import re
keyword_list_new=[]
keyword_list = set(email_words)
for item in keyword_list:
    is_re_list = False
    for pattern1 in re_pattern_list:
        result=re.findall(pattern1, item)
        if  len(result) > 0:
            is_re_list = True
            break
    if is_re_list == False:
        keyword_list_new.append(item)
len(keyword_list_new)

4644

## compare corpus with n-gram keywords, n=2~4

In [130]:
# check if key word is found 
def check_special_term(words, start_index, n_gram):
    merge_word=[]
    for i in range(n_gram):
        merge_word.append(words[start_index+i])
    if ' '.join(merge_word) in keyword_list_TERM:
        #print('converted_word=', keyword_list_TERM[' '.join(merge_word)]['converted_word'])
        return keyword_list_TERM[' '.join(merge_word)]['converted_word']
    else:
        return ''

# scan corpus and replace key word by converted_word, which use underscope instead of space     
def compare_corpus_with_keyword(clean_corpus):
    new_clean_corpus=[]
    merge_word_list=[] # n-gram keywords
    hit_row_index=[] # keep for predict test
    for no, line in enumerate(clean_corpus):
        words = line.split(' ')
        #words = word_tokenize(line) 
        words_len = len(words)
        # n-gram
        new_line=''
        for i in range(4, 1, -1):
            for j in range(words_len-i+1):
                merge_word = check_special_term(words, j, i)
                if merge_word == '':
                    new_line+=' '+words[j]
                    if j==words_len-i:
                        for k in range(j+1, words_len):
                            new_line+=' '+words[k]
                    continue
                else:
                    merge_word_list.append(merge_word)
                    hit_row_index.append(no)
                    new_line+=' '+merge_word
                    j+=i
                    continue

            # line is too short, keep it as original
            if words_len-i<0:
                new_line=line    
            new_line = new_line.strip()
            words = new_line.split(' ')
            words_len = len(words)
            if i>2 and words != 'kb':
                new_line=''
        new_clean_corpus.append(new_line)
        #if new_line != line:
        #    print('org_line='+line+'\n')
        #    print('new_line='+new_line+'\n\n')
    return new_clean_corpus, merge_word_list, hit_row_index

new_clean_corpus, merge_word_list, hit_row_index = compare_corpus_with_keyword(clean_corpus)
hit_row_index=list(set(hit_row_index))        

## Calculate BOW

In [131]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
BOW_vector = vectorizer.fit_transform(new_clean_corpus)
X = BOW_vector.toarray()

vocab = vectorizer.get_feature_names()
pickle.dump(vectorizer.vocabulary_,open("feature.pkl","wb"))
print('BOW={}, no. of vocabulary={}'.format(X.shape, len(vocab)))

word2id = dict((v, idx) for idx, v in enumerate(vocab))
pickle.dump(word2id,open("word2id.pkl","wb"))


BOW=(34012, 4640), no. of vocabulary=4640


## Create dictionary from vocabulary

In [132]:
# vectorizer.vocabulary_

In [133]:
import numpy as np
import guidedlda

#X = guidedlda.datasets.load_data(guidedlda.datasets.NYT)
#vocab = guidedlda.datasets.load_vocab(guidedlda.datasets.NYT)


## 加重特定單字在某些Topics的權重

In [134]:
# Guided LDA with seed topics.

model = guidedlda.GuidedLDA(n_topics=n_topics, n_iter=100, refresh=20)
seed_topics = {}
for t_id, st in enumerate(seed_topic_list):
    for word in st:
        if not word in word2id.keys():
            word2id[word] = t_id

        seed_topics[word2id[word]] = t_id
model.fit(X, seed_topics=seed_topics, seed_confidence=0.15)


INFO:guidedlda:n_documents: 34012
INFO:guidedlda:vocab_size: 4640
INFO:guidedlda:n_words: 116617
INFO:guidedlda:n_topics: 10
INFO:guidedlda:n_iter: 100
INFO:guidedlda:<0> log likelihood: -1234640
INFO:guidedlda:<20> log likelihood: -768666
INFO:guidedlda:<40> log likelihood: -728593
INFO:guidedlda:<60> log likelihood: -706512
INFO:guidedlda:<80> log likelihood: -695612
INFO:guidedlda:<99> log likelihood: -689564


## 顯示每個Topics前八個重要的單字

In [135]:
n_top_words = 10
topic_word = model.topic_word_
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))
    print('Keywords: {}'.format(seed_topic_list[i]))


Topic 0: inquiry product upgrade wfb url version configuration exclusion issue deploy
Keywords: ['configuration', 'network_drive', 'cisco', 'vpn', 'firewall', 'active_directory']
Topic 1: file false block positive application program encryption unauthorized detect detection
Keywords: ['false_positive', 'url_reclassification', 'whitelist', 'white_listing', 'block', 'unauthorized_encryption']
Topic 2: agent console access web_console offline security server unable install upgrade
Keywords: ['sa_offline', 'corruption', 'behavior_monitor', 'web_console']
Topic 3: console block access email unable error agent url page web_console
Keywords: ['compatibility', '3rd_party', 'update', 'mcafee', 'msa', 'exchange_server', 'quickbook', 'office']
Topic 4: agent installation install security issue error unable deploy server mac
Keywords: ['performance', 'high_cpu', 'crash', 'hang', 'slowness', 'office', 'bsod', '3rd_party']
Topic 5: license inquiry activate account seat product issue renewal code cha

In [136]:
# test
'white_listing' in vocab

False

## 顯示每個Document所屬的Topics

In [137]:
display_document_count=20
display_keyword_count_per_topic=5

doc_topic = model.transform(X)
for i in range(display_document_count):
    print('原文:', df.loc[i, 'Subject'])
    doc=[]
    #vocab_len = len(vocab)
    #for j in range(vocab_len):
    #    if X[i, j] != 0:
    #        doc.append(vocab[j])
    #print(' '.join(doc))
    
    print("top topic: {}".format(doc_topic[i].argmax()))
    top_n_words = X[i,:].argsort()[:display_keyword_count_per_topic:-1]
    top_n_words_list = []
    for k in top_n_words:
        if X[i,k] > 0:
            top_n_words_list.append(k)
    print("Document: {}".format(', '.join(np.array(vocab)[top_n_words_list])))
    #print("Document: {}".format(', '.join(np.array(vocab)[list(reversed(X[i,:].argsort()))[:display_keyword_count_per_topic:-1]])))
    print(' ')

原文: [WFBS-SVC] Installation on Mac Is Not Full Featured
top topic: 7
Document: mac, full, feature, installation
 
原文: [WFBS - A] Smart Scan is not updating
top topic: 7
Document: smart, scan, update
 
原文: [WFBS - SVC] Installation issue
top topic: 4
Document: issue, installation
 
原文: [MALWARE][WFBS SVC] Wallet Ransomware
top topic: 9
Document: ransomware, wallet
 
原文: [MALWARE][WFBS S 9.5]Possible Ransomware detection on clients machine
top topic: 9
Document: machine, client, possible, detection, wfb, ransomware
 
原文: [WFBS-S] Adding Exclusion to Behavior Monitoring
top topic: 1
Document: add, behavior_monitor, exclusion
 
原文: [WFBS-S] Migrating from An Old Server to A New Server
top topic: 6
Document: server, old, migrate, new
 
原文: [WFBS-SVC] Agent cloning
top topic: 4
Document: agent, clone
 
原文: [WFBS-SVC] Main Software not Functioning with Trend Micro Security Agent
top topic: 4
Document: main, function, security, micro, agent, software
 
原文: [WFBS-SVC] False Positive: Applicatio

## lighten the model object

In [138]:
# This step will delete some matrices inside the model.
model.purge_extra_matrices()

## Save the model 

In [139]:
from six.moves import cPickle as pickle
with open('guidedlda_model.pickle', 'wb') as file_handle:
    pickle.dump(model, file_handle)